In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from ast import literal_eval
import matplotlib.pyplot as plt

In [22]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Content Recommender

In [23]:
links_small = pd.read_csv('data/input/movies/links_small.csv')
md = pd.read_csv('data/input/movies/movies_metadata.csv')

/var/folders/4v/r4sjb8m14md8bj477_q2b9s00000gn/T/ipykernel_59655/2262967869.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('data/input/movies/movies_metadata.csv')


In [24]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])

**Creating a smaller dataset to work with**

In [25]:
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)].copy()
smd.shape

(9099, 24)

In [26]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [29]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [30]:
tfidf_matrix.shape

(9099, 268124)

**Cosine Similarity**

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. Mathematically, it is defined as follows

In [31]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)